In [1]:
import Module.Conn as conn 
import Module.GetDataSet as getdataset 
import pandas as pd 
import numpy as np 
import random
import copy 
import math 
from collections import Counter
from matplotlib import pyplot as plt
import cv2
from sklearn.metrics import accuracy_score,f1_score
from Module.DotClass import *
from Module.GetDistance import *
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False

random.seed(0)

In [2]:
standard_df=getdataset.get_standard_dataset()

In [3]:
# standard_df.trajectory.apply(lambda x:len(eval(x))).max()
# x_min=standard_df.trajectory.apply(lambda x:pd.DataFrame(eval(x))['ab_x'].min()).min()
# x_max=standard_df.trajectory.apply(lambda x:pd.DataFrame(eval(x))['ab_x'].max()).max()
# y_min=standard_df.trajectory.apply(lambda x:pd.DataFrame(eval(x))['ab_y'].min()).min()
# y_max=standard_df.trajectory.apply(lambda x:pd.DataFrame(eval(x))['ab_y'].max()).max()

In [4]:
# print(x_min,x_max,y_min,y_max)
#7.917125697412757 80.92179799639905 29.749954279653075 101.04029382952234

In [5]:
print('字数:',len(set(standard_df.id)))
# standard_df 

字数: 2416


In [6]:
#构造单个错误数据集
#先聚合成整字为单位
standard_df1=standard_df.copy()
total_xy=[]
for j in range(len(standard_df1)):
    t=standard_df1.trajectory.iloc[j]
    xy=[]
    for i in eval(t):
        xy.append([i['ab_x'],i['ab_y']])
    total_xy.append(xy)
standard_df1["total_xy"]=total_xy
standard_df1

#total_xy填充成348维

total_xy=standard_df1.groupby(['id'])['total_xy'].apply(lambda x:list(x))
labels=standard_df1.groupby(['id'])['label'].apply(lambda x:list(x))
chs=standard_df1.groupby(['id'])['ch'].apply(lambda x:list(x)[0])
t=total_xy.to_frame().join(labels.to_frame()).join(chs.to_frame())
df_backup=t.reset_index()
df=df_backup.copy(deep=True)
df.head()

,id,total_xy,label,ch
0,606be05bd02f079e979acea1,"[[[27.29623646957242, 47.41327858822196], [27....","[点, 横折提, 横, 竖, 横, 横, 竖钩, 点]",诗
1,606be09bd02f079e979acea2,"[[[15.208819478150502, 64.17993234978343], [15...","[横, 竖, 撇, 点, 横, 竖钩, 点]",村
2,606be0c1d02f079e979acea3,"[[[38.818714142500966, 36.87958841743438], [38...","[点, 横, 点, 撇, 横, 竖, 横折, 横, 横, 竖, 横, 横]",童
3,606be1259d76cefbb8d23eba,"[[[23.487325167875582, 45.930255238761276], [2...","[横, 横, 竖, 提, 撇, 竖, 横折, 横, 横, 横, 撇, 竖, 横折, 横]",碧
4,606c0c65e00923b7467b5ad6,"[[[29.070172446316946, 52.25796739800715], [29...","[横, 横, 横, 撇, 捺, 竖, 横折, 横, 横]",春


# 多笔画

In [7]:
sample_size=200
# 在总体数据集上构造错误数据集
#new_total_xy,new_label,error_type[],error_tag[],序号都从0开始
#先构造单个笔画错误

rand_seed=0
# 多笔画-断笔-挑选一笔，折断
#从原始df中挑选sample_size条数据
random.seed(rand_seed)
rand_seed+=1
idx_list_more_split=random.sample([i for i in range(len(df))], sample_size)#随机选取200个idx

idx=0
df_item_list_more_split=[]
for idx in idx_list_more_split:
    t=df.iloc[idx]
    t_label=copy.deepcopy(t.label)
    t_total_xy=copy.deepcopy(t.total_xy)#可以在上面修改，不会改变df的内容
    random.seed(rand_seed)
    rand_seed+=1
    seq_idx=random.randint(0,len(t_label)-1) #挑选一个笔画
    t_xy=t_total_xy[seq_idx]#400*2
    cut_dot_seq_idx_m=random.randint(int(len(t_xy)*0.3),int(len(t_xy)*0.7))#挑选截取位置#一个笔画序列30%到70%的地方截断，截取10%的内容
    cut_dot_seq_len=random.randint(0,int(len(t_xy)*0.05))
    cut_dot_seq_idx_s,cut_dot_seq_idx_e=cut_dot_seq_idx_m-cut_dot_seq_len,cut_dot_seq_idx_m+cut_dot_seq_len#挑选截取位置#一个笔画序列20%到80%的地方截断，截取10%的内容
    new_xy_1=t_xy[:cut_dot_seq_idx_s]
    new_xy_2=t_xy[cut_dot_seq_idx_e:]
    t_new_total_xy=t_total_xy[:seq_idx]+[new_xy_1]+[new_xy_2]+t_total_xy[seq_idx+1:]
    t_new_label=t_label[:seq_idx]+[t_label[seq_idx]]+[t_label[seq_idx]]+t_label[seq_idx+1:]
    t_new_seq=[i for i in range(len(t_label))]
    t_new_seq.insert(seq_idx,t_new_seq[seq_idx])
    df_item={"id":t.id,"total_xy":t.total_xy,"label":t.label,"ch":t.ch,"new_total_xy":t_new_total_xy,"new_label":t_new_label,"new_seq":t_new_seq,"error_type":["more"],"error_tag":[('more','split',(seq_idx,seq_idx+1))]}
    df_item_list_more_split.append(df_item)
df_more_split=pd.DataFrame(df_item_list_more_split)

# 多笔画-插入一个不相关的笔画
random.seed(rand_seed)
rand_seed+=1
idx_list_more_nonsplit=random.sample([i for i in range(len(df))], sample_size)#随机选取200个idx

df_item_list_more_nonsplit=[]
for idx in idx_list_more_nonsplit:
    t=df.iloc[idx]
    t_label=copy.deepcopy(t.label)
    t_total_xy=copy.deepcopy(t.total_xy)#可以在上面修改，不会改变df的内容
    random.seed(rand_seed)
    rand_seed+=1
    seq_idx=random.randint(0,len(t_label)-1) #挑选一个笔画
    t_xy=t_total_xy[seq_idx]#400*2

    #准备其他汉字的某一笔
    other_idx_list=list(df[df.ch!=t.ch].index)#非相同汉字
    idx_other=random.sample(other_idx_list, 1)[0]
    t_other=df.iloc[idx_other]
    other_seq_idx=random.randint(0,len(t_other.label)-1)#这里没有限制非相同笔画
    t_insert_xy=t_other.total_xy[other_seq_idx]
    t_insert_label=t_other.label[other_seq_idx]

    t_new_total_xy=t_total_xy[:seq_idx]+[t_insert_xy]+t_total_xy[seq_idx:]
    t_new_label=t_label[:seq_idx]+[t_insert_label]+t_label[seq_idx:]
    t_new_seq=[i for i in range(len(t_label))]
    t_new_seq.insert(seq_idx,-1)
    df_item={"id":t.id,"total_xy":t.total_xy,"label":t.label,"ch":t.ch,"new_total_xy":t_new_total_xy,"new_label":t_new_label,"new_seq":t_new_seq,"error_type":["more"],"error_tag":[('more','nonsplit',seq_idx)]}
    df_item_list_more_nonsplit.append(df_item)
df_more_nonsplit=pd.DataFrame(df_item_list_more_nonsplit)
df_more_nonsplit

# pd.concat([df_more_split,df_more_nonsplit])

,id,total_xy,label,ch,new_total_xy,new_label,new_seq,error_type,error_tag
0,606c0ed03717b67800102ced,"[[[31.75539062528776, 43.64542095575307], [32....","[撇折, 撇折, 提, 竖, 横折, 横, 横, 横]",组,"[[[31.75539062528776, 43.64542095575307], [32....","[撇折, 撇折, 提, 竖, 横折, 横, 竖, 横, 横]","[0, 1, 2, 3, 4, 5, -1, 6, 7]",[more],"[(more, nonsplit, 6)]"
1,6188a83487f3af50f2e803c8,"[[[31.473878001090895, 42.500036438510215], [3...","[横, 点, 横撇/横钩, 竖, 点, 点, 点, 点, 撇, 横折钩, 横折, 横, 竖弯钩]",雹,"[[[31.473878001090895, 42.500036438510215], [3...","[横, 横, 点, 横撇/横钩, 竖, 点, 点, 点, 点, 撇, 横折钩, 横折, 横,...","[0, -1, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]",[more],"[(more, nonsplit, 1)]"
2,61c3cb02c0766b5221b53959,"[[[33.334774316568286, 46.23195139185694], [34...","[点, 点, 撇, 横, 撇, 捺, 横, 横, 竖]",举,"[[[33.334774316568286, 46.23195139185694], [34...","[点, 点, 撇, 横, 撇, 捺, 撇点, 横, 横, 竖]","[0, 1, 2, 3, 4, 5, -1, 6, 7, 8]",[more],"[(more, nonsplit, 6)]"
3,606c0c76e00923b7467b5ae5,"[[[37.652179743393106, 44.01837440939644], [37...","[撇点, 撇, 横, 撇, 横, 横, 竖, 横]",姓,"[[[37.652179743393106, 44.01837440939644], [37...","[撇点, 撇, 横, 撇, 横, 横, 横, 竖, 横]","[0, 1, 2, 3, 4, 5, -1, 6, 7]",[more],"[(more, nonsplit, 6)]"
4,6184bbc587f3af50f2e80310,"[[[41.63856530017486, 38.89003196783734], [41....","[撇, 横折弯钩/横斜钩, 横, 竖钩, 撇, 点]",杂,"[[[29.5630928700827, 77.71144113192192], [29.8...","[横, 撇, 横折弯钩/横斜钩, 横, 竖钩, 撇, 点]","[-1, 0, 1, 2, 3, 4, 5]",[more],"[(more, nonsplit, 0)]"
...,...,...,...,...,...,...,...,...,...
195,606c0fc84c1e167a0cc38c95,"[[[31.64303144218843, 47.11670831352215], [31....","[横折, 横, 横, 点, 撇, 撇, 捺]",灵,"[[[43.05810469162179, 51.024572211871806], [43...","[撇, 横折, 横, 横, 点, 撇, 撇, 捺]","[-1, 0, 1, 2, 3, 4, 5, 6]",[more],"[(more, nonsplit, 0)]"
196,61c684b905bb4801b55d35e8,"[[[30.070317550271227, 42.60079471056895], [30...","[撇, 横, 横, 横, 竖提, 撇, 横折钩, 撇折, 点]",钩,"[[[30.070317550271227, 42.60079471056895], [30...","[撇, 横, 横, 横, 竖提, 撇, 点, 横折钩, 撇折, 点]","[0, 1, 2, 3, 4, 5, -1, 6, 7, 8]",[more],"[(more, nonsplit, 6)]"
197,61c9986305bb4801b55d36c0,"[[[36.96495666960311, 30.996064380095998], [37...","[点, 点, 横撇/横钩, 撇, 点, 撇, 横, 竖, 横, 横]",窄,"[[[21.784083813561864, 56.16273167682101], [22...","[点, 点, 点, 横撇/横钩, 撇, 点, 撇, 横, 竖, 横, 横]","[-1, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9]",[more],"[(more, nonsplit, 0)]"
198,61ca646505bb4801b55d36d8,"[[[38.72534836381243, 46.41132095419905], [38....","[撇, 竖, 横折, 横, 横]",白,"[[[38.72534836381243, 46.41132095419905], [38....","[撇, 竖, 横折弯钩/横斜钩, 横折, 横, 横]","[0, 1, -1, 2, 3, 4]",[more],"[(more, nonsplit, 2)]"


# 少笔画

In [8]:
#少笔画-连笔-直接连接某连续两笔
# rand_seed=0
random.seed(rand_seed)
rand_seed+=1
mylist=[]#去掉只有一个笔画的汉字
for i in range(len(df.label)):
    if len(df.label.iloc[i])>1:
        mylist.append(i)
idx_list_less_merge=random.sample(mylist, sample_size)#随机选取200个idx

idx=0
df_item_list_less_merge=[]
for idx in idx_list_less_merge:
    t=df.iloc[idx]
    t_label=copy.deepcopy(t.label)
    t_total_xy=copy.deepcopy(t.total_xy)#可以在上面修改，不会改变df的内容
    random.seed(rand_seed)
    rand_seed+=1
    seq_idx=random.randint(0,len(t_label)-2) #挑选一个笔画，合并这个笔画和之后一笔
    t_xy=t_total_xy[seq_idx]+t_total_xy[seq_idx+1]
    t_new_total_xy=t_total_xy[:seq_idx]+[t_xy]+t_total_xy[seq_idx+2:]
    t_new_label=t_label[:seq_idx]+[(t_label[seq_idx],t_label[seq_idx+1])]+t_label[seq_idx+2:]
    t_new_seq=[i for i in range(len(t_label))]
    t_new_seq[seq_idx]=(-1)
    t_new_seq.pop(seq_idx+1)
    df_item={"id":t.id,"total_xy":t.total_xy,"label":t.label,"ch":t.ch,"new_total_xy":t_new_total_xy,"new_label":t_new_label,"new_seq":t_new_seq,"error_type":["less"],"error_tag":[('less','merge',(seq_idx,seq_idx+1))]}
    df_item_list_less_merge.append(df_item)
df_less_merge=pd.DataFrame(df_item_list_less_merge)
df_less_merge

# #少笔画-随机减少一笔
random.seed(rand_seed)
rand_seed+=1
idx_list_less_nonmerge=random.sample(mylist, sample_size)#随机选取200个idx

idx=0
df_item_list_less_nonmerge=[]
for idx in idx_list_less_merge:
    t=df.iloc[idx]
    t_label=copy.deepcopy(t.label)
    t_total_xy=copy.deepcopy(t.total_xy)#可以在上面修改，不会改变df的内容
    random.seed(rand_seed)
    rand_seed+=1
    seq_idx=random.randint(0,len(t_label)-1) #挑选一个笔画
    t_new_total_xy=t_total_xy[:seq_idx]+t_total_xy[seq_idx+1:]
    t_new_label=t_label[:seq_idx]+t_label[seq_idx+1:]
    t_new_seq=[i for i in range(len(t_label))]
    t_new_seq.pop(seq_idx)
    df_item={"id":t.id,"total_xy":t.total_xy,"label":t.label,"ch":t.ch,"new_total_xy":t_new_total_xy,"new_label":t_new_label,"new_seq":t_new_seq,"error_type":["less"],"error_tag":[('less','nonmerge',seq_idx)]}
    df_item_list_less_nonmerge.append(df_item)
df_less_nonmerge=pd.DataFrame(df_item_list_less_nonmerge)
# df_less_nonmerge
# pd.concat([df_less_merge,df_less_nonmerge])

# 错笔画

In [9]:
# 错笔画-平移
rand_seed=0
random.seed(rand_seed)
rand_seed+=1
idx_list_wrong_parallel=random.sample([i for i in range(len(df))], sample_size)#随机选取200个idx

df_item_list_wrong_parallel=[]
for idx in idx_list_wrong_parallel:#选一个字
#     idx=idx_list_wrong_parallel[0]
    t=df.iloc[idx]
    t_label=copy.deepcopy(t.label)
    t_total_xy=copy.deepcopy(t.total_xy)#可以在上面修改，不会改变df的内容
    random.seed(rand_seed)
    rand_seed+=1
    seq_idx=random.randint(0,len(t_label)-1) #挑选一个笔画
    t_xy=t_total_xy[seq_idx]#对它做平移变换
    rand_x_shift,rand_y_shift=random.uniform(10,80-t_xy[0][0])*pow(-1,random.randint(0,1)),random.uniform(10,80-t_xy[0][1])*pow(-1,random.randint(1,2))
    t_xy=np.array(t_xy)+np.array(rand_x_shift)
    t_xy=[[i for i in j] for j in t_xy]#类型转换
    t_new_total_xy=t_total_xy[:seq_idx]+[t_xy]+t_total_xy[seq_idx+1:]
    t_new_label=t_label[:seq_idx]+[t_label[seq_idx]]+t_label[seq_idx+1:]
    t_new_seq=[i for i in range(len(t_label))]
    t_new_seq[seq_idx]=-1
    df_item={"id":t.id,"total_xy":t.total_xy,"label":t.label,"ch":t.ch,"new_total_xy":t_new_total_xy,"new_label":t_new_label,"new_seq":t_new_seq,"error_type":["wrong"],"error_tag":[('wrong','parallel',seq_idx)]}
    df_item_list_wrong_parallel.append(df_item)
df_wrong_parallel=pd.DataFrame(df_item_list_wrong_parallel)
df_wrong_parallel

#错笔画-旋转
random.seed(rand_seed)
rand_seed+=1
idx_list_wrong_rotation=random.sample([i for i in range(len(df))], sample_size)#随机选取200个idx

df_item_list_wrong_rotation=[]
for idx in idx_list_wrong_rotation:#选一个字
#     idx=idx_list_wrong_rotation[0]
    t=df.iloc[idx]
    t_label=copy.deepcopy(t.label)
    t_total_xy=copy.deepcopy(t.total_xy)#可以在上面修改，不会改变df的内容
    random.seed(rand_seed)
    rand_seed+=1
    seq_idx=random.randint(0,len(t_label)-1) #挑选一个笔画
    t_xy=np.array(t_total_xy[seq_idx])#对它做平移变换
    xy0=t_xy[random.randint(0,len(t_xy)-1)]#随机选中笔画中的某一个点，作为中心点
    theta = np.random.uniform(90/180*np.pi,180/180*np.pi)*pow(-1,random.randint(0,1))
    rotation_matrix = np.array([[np.cos(theta), -np.sin(theta)],[np.sin(theta), np.cos(theta)]])
    t_xy = (t_xy-xy0).dot(rotation_matrix) +xy0 # random rotation
    t_xy=[[i for i in j] for j in t_xy]
    t_new_total_xy=t_total_xy[:seq_idx]+[t_xy]+t_total_xy[seq_idx+1:]
    t_new_label=t_label[:seq_idx]+[t_label[seq_idx]]+t_label[seq_idx+1:]
    t_new_seq=[i for i in range(len(t_label))]
    t_new_seq[seq_idx]=-1
    df_item={"id":t.id,"total_xy":t.total_xy,"label":t.label,"ch":t.ch,"new_total_xy":t_new_total_xy,"new_label":t_new_label,"new_seq":t_new_seq,"error_type":["wrong"],"error_tag":[('wrong','rotation',seq_idx)]}
    df_item_list_wrong_rotation.append(df_item)
df_wrong_rotation=pd.DataFrame(df_item_list_wrong_rotation)
df_wrong_rotation


#错笔画-类别错误，替换成别的字里面的不同笔画

random.seed(rand_seed)
rand_seed+=1
mylist=[]#去掉只有一个笔画的汉字
for i in range(len(df.label)):
    if len(df.label.iloc[i])>1:
        mylist.append(i)
idx_list_wrong_type=random.sample([i for i in range(len(df))], sample_size)#随机选取200个idx

df_item_list_wrong_type=[]
for idx in idx_list_wrong_type:
    t=df.iloc[idx]
    t_label=copy.deepcopy(t.label)
    t_total_xy=copy.deepcopy(t.total_xy)#可以在上面修改，不会改变df的内容
    random.seed(rand_seed)
    rand_seed+=1
    seq_idx=random.randint(0,len(t_label)-1) #挑选一个笔画
    t_xy=t_total_xy[seq_idx]#400*2

    #准备其他汉字的某一笔
    other_idx_list=list(df[df.ch!=t.ch].index)#非相同汉字
    t_list=list(set(mylist)&set(other_idx_list))#且汉字笔画数大于1
    idx_other=random.sample(t_list, 1)[0]
    t_other=df.iloc[idx_other]
    checklist=[]
    for i in range(len(t_other.label)):
        if t_other.label[i]!=t_label[seq_idx]:#这里限制非相同笔画
            checklist.append(i)
    other_seq_idx=random.sample(checklist, 1)[0]
    t_insert_xy=copy.deepcopy(t_other.total_xy[other_seq_idx])
    t_insert_label=t_other.label[other_seq_idx]
    
    t_insert_xy=np.array(t_insert_xy)-np.array(t_insert_xy[0])+np.array(t_xy[0])#平移到当前笔画这里
    t_insert_xy=[[i for i in j] for j in t_insert_xy]
    t_new_total_xy=t_total_xy[:seq_idx]+[t_insert_xy]+t_total_xy[seq_idx+1:]
    t_new_label=t_label[:seq_idx]+[t_insert_label]+t_label[seq_idx+1:]
    t_new_seq=[i for i in range(len(t_label))]
    t_new_seq[seq_idx]=-1
    df_item={"id":t.id,"total_xy":t.total_xy,"label":t.label,"ch":t.ch,"new_total_xy":t_new_total_xy,"new_label":t_new_label,"new_seq":t_new_seq,"error_type":["wrong"],"error_tag":[('wrong','type',seq_idx)]}
    df_item_list_wrong_type.append(df_item)
df_wrong_type=pd.DataFrame(df_item_list_wrong_type)
df_wrong_type




# 错笔画-长短错误
random.seed(rand_seed)
rand_seed+=1
idx_list_wrong_size=random.sample([i for i in range(len(df))], sample_size)#随机选取200个idx

df_item_list_wrong_size=[]
for idx in idx_list_wrong_size:#选一个字
    t=df.iloc[idx]
    t_label=copy.deepcopy(t.label)
    t_total_xy=copy.deepcopy(t.total_xy)#可以在上面修改，不会改变df的内容
    random.seed(rand_seed)
    rand_seed+=1
    seq_idx=random.randint(0,len(t_label)-1) #挑选一个笔画
    t_xy=np.array(t_total_xy[seq_idx])#对它做变换
    if rand_seed%2==0:
        ratio = np.random.uniform(0.1,0.5)
    else:
        ratio = np.random.uniform(1.5,1.9)
    t_xy=(t_xy-t_xy[0])*ratio+t_xy[0]
#     t_xy = (t_xy-xy0).dot(rotation_matrix) +xy0 # random rotation
    t_xy=[[i for i in j] for j in t_xy]
    t_new_total_xy=t_total_xy[:seq_idx]+[t_xy]+t_total_xy[seq_idx+1:]
    t_new_label=t_label[:seq_idx]+[t_label[seq_idx]]+t_label[seq_idx+1:]
    t_new_seq=[i for i in range(len(t_label))]
    t_new_seq[seq_idx]=-1
    df_item={"id":t.id,"total_xy":t.total_xy,"label":t.label,"ch":t.ch,"new_total_xy":t_new_total_xy,"new_label":t_new_label,"new_seq":t_new_seq,"error_type":["wrong"],"error_tag":[('wrong','size',seq_idx)]}
    df_item_list_wrong_size.append(df_item)
df_wrong_size=pd.DataFrame(df_item_list_wrong_size)
df_wrong_size
#

,id,total_xy,label,ch,new_total_xy,new_label,new_seq,error_type,error_tag
0,6107df56a11a58c6dbb96bc2,"[[[27.111068063065233, 44.213173735376834], [2...","[点, 横折提, 撇, 捺, 撇, 竖弯钩]",论,"[[[27.111068063065233, 44.213173735376834], [2...","[点, 横折提, 撇, 捺, 撇, 竖弯钩]","[-1, 1, 2, 3, 4, 5]",[wrong],"[(wrong, size, 0)]"
1,61ca64b005bb4801b55d36dd,"[[[40.60415207945874, 47.370266259886634], [40...","[撇, 捺]",人,"[[[40.60415207945874, 47.370266259886634], [40...","[撇, 捺]","[-1, 1]",[wrong],"[(wrong, size, 0)]"
2,61ca6bd705bb4801b55d3729,"[[[33.78337975532861, 43.05731276182386], [33....","[撇, 竖, 竖, 横折, 横, 横, 横]",但,"[[[33.78337975532861, 43.05731276182386], [33....","[撇, 竖, 竖, 横折, 横, 横, 横]","[0, 1, 2, 3, 4, -1, 6]",[wrong],"[(wrong, size, 5)]"
3,6188a77787f3af50f2e803c6,"[[[34.47982558001581, 43.6580380330734], [34.7...","[横, 点, 横撇/横钩, 竖, 点, 点, 点, 点, 撇, 捺, 点, 横撇/横钩, 点]",零,"[[[34.47982558001581, 43.6580380330734], [34.7...","[横, 点, 横撇/横钩, 竖, 点, 点, 点, 点, 撇, 捺, 点, 横撇/横钩, 点]","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, -1, 12]",[wrong],"[(wrong, size, 11)]"
4,61c40bc6c0766b5221b539ca,"[[[35.90055328184522, 49.45422077037556], [36....","[横折, 横, 横, 竖提, 撇, 点, 点, 横折折撇, 捺]",退,"[[[35.90055328184522, 49.45422077037556], [36....","[横折, 横, 横, 竖提, 撇, 点, 点, 横折折撇, 捺]","[0, 1, -1, 3, 4, 5, 6, 7, 8]",[wrong],"[(wrong, size, 2)]"
...,...,...,...,...,...,...,...,...,...
195,61c68a6e05bb4801b55d3614,"[[[16.249511767902362, 61.09412002594685], [16...","[横, 竖钩, 提, 点, 点, 横撇/横钩, 横, 竖钩]",拧,"[[[16.249511767902362, 61.09412002594685], [16...","[横, 竖钩, 提, 点, 点, 横撇/横钩, 横, 竖钩]","[0, 1, 2, -1, 4, 5, 6, 7]",[wrong],"[(wrong, size, 3)]"
196,61c2b4c4c0766b5221b5392e,"[[[16.38510584795631, 62.15914693232094], [16....","[横, 竖, 提, 点, 横, 撇, 横折, 横, 横, 竖, 竖, 横折, 横]",塘,"[[[16.38510584795631, 62.15914693232094], [16....","[横, 竖, 提, 点, 横, 撇, 横折, 横, 横, 竖, 竖, 横折, 横]","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, -1, 11, 12]",[wrong],"[(wrong, size, 10)]"
197,61c67be105bb4801b55d35bd,"[[[36.19713363683579, 39.679051492729776], [36...","[点, 横, 竖, 横折, 横, 竖, 撇, 竖提, 撇, 捺]",衷,"[[[36.19713363683579, 39.679051492729776], [36...","[点, 横, 竖, 横折, 横, 竖, 撇, 竖提, 撇, 捺]","[0, -1, 2, 3, 4, 5, 6, 7, 8, 9]",[wrong],"[(wrong, size, 1)]"
198,6186272a87f3af50f2e803ab,"[[[21.54900475532861, 50.70150700201999], [21....","[横, 横, 横, 撇, 横折折折钩/横撇弯钩, 竖]",邦,"[[[21.54900475532861, 50.70150700201999], [21....","[横, 横, 横, 撇, 横折折折钩/横撇弯钩, 竖]","[0, 1, 2, 3, -1, 5]",[wrong],"[(wrong, size, 4)]"


# 笔顺错误

In [10]:
#笔顺错误-非同类笔画错误
random.seed(rand_seed)
rand_seed+=1
mylist=[]#去掉只有一类笔画的汉字
for i in range(len(df.label)):
    if len(set(df.label.iloc[i]))>1:
        mylist.append(i)
idx_list_exchange_different=random.sample(mylist, sample_size)#随机选取200个idx

df_item_list_exchange_different=[]
for idx in idx_list_exchange_different:
#     t=df.iloc[idx]
    t_label=copy.deepcopy(t.label)
    t_total_xy=copy.deepcopy(t.total_xy)#可以在上面修改，不会改变df的内容
    random.seed(rand_seed)
    rand_seed+=1
    seq_idx=random.randint(0,len(t_label)-1) #挑选一个笔画
    t_xy=copy.deepcopy(t_total_xy[seq_idx])#400*2
    t_label_0=copy.deepcopy(t_label[seq_idx])
    #准备该汉字的其他某一笔
    checklist=[]
    for i in range(len(t_label)):
        if t_label[i]!=t_label[seq_idx]:#这里限制非相同笔画
            checklist.append(i)
    other_idx=random.sample(checklist, 1)[0]
    t_total_xy[seq_idx]=t_total_xy[other_idx]
    t_total_xy[other_idx]=t_xy
    t_label[seq_idx]=t_label[other_idx]
    t_label[other_idx]=t_label_0
    t_new_total_xy=t_total_xy
    t_new_label=t_label
    t_new_seq=[i for i in range(len(t_label))]
    t_seq=t_new_seq[seq_idx]
    t_new_seq[seq_idx]=t_new_seq[other_idx]
    t_new_seq[other_idx]=t_seq
    df_item={"id":t.id,"total_xy":t.total_xy,"label":t.label,"ch":t.ch,"new_total_xy":t_new_total_xy,"new_label":t_new_label,"new_seq":t_new_seq,"error_type":["exchange"],"error_tag":[('exchange','different',(min(seq_idx,other_idx),max(seq_idx,other_idx)))]}
    df_item_list_exchange_different.append(df_item)
df_exchange_different=pd.DataFrame(df_item_list_exchange_different)
df_exchange_different

#笔顺错误-同类笔画错误
random.seed(rand_seed)
rand_seed+=1
mylist=[]#去掉只有一类笔画的汉字
for i in range(len(df.label)):
    if len(set(df.label.iloc[i]))<len(df.label.iloc[i]):#找到有重复笔画的字
        mylist.append(i)
idx_list_exchange_same=random.sample(mylist, sample_size)#随机选取200个idx

df_item_list_exchange_same=[]
for idx in idx_list_exchange_same:
    t=df.iloc[idx]
    t_label=copy.deepcopy(t.label)
    t_total_xy=copy.deepcopy(t.total_xy)#可以在上面修改，不会改变df的内容
    random.seed(rand_seed)
    rand_seed+=1
    counter = Counter(t_label)
    mydic=dict(counter)
    checklist=[]
    for i in range(len(t_label)):
        if mydic[t_label[i]]>1:#这里限制非相同笔画
            checklist.append(i)
    seq_idx=random.sample(checklist, 1)[0]
    t_xy=copy.deepcopy(t_total_xy[seq_idx])#400*2
    t_label_0=copy.deepcopy(t_label[seq_idx])
    #准备该汉字的其他某一笔
    checklist=[]
    for i in range(len(t_label)):
        if t_label[i]==t_label[seq_idx]:#这里限制非相同笔画
            checklist.append(i)
    checklist=[i for i in checklist if i!=seq_idx]
    other_idx=random.sample(checklist, 1)[0]
    t_total_xy[seq_idx]=t_total_xy[other_idx]
    t_total_xy[other_idx]=t_xy
    t_label[seq_idx]=t_label[other_idx]
    t_label[other_idx]=t_label_0
    t_new_total_xy=t_total_xy
    t_new_label=t_label
    t_new_seq=[i for i in range(len(t_label))]
    t_seq=t_new_seq[seq_idx]
    t_new_seq[seq_idx]=t_new_seq[other_idx]
    t_new_seq[other_idx]=t_seq
    df_item={"id":t.id,"total_xy":t.total_xy,"label":t.label,"ch":t.ch,"new_total_xy":t_new_total_xy,"new_label":t_new_label,"new_seq":t_new_seq,"error_type":["exchange"],"error_tag":[('exchange','same',(min(seq_idx,other_idx),max(seq_idx,other_idx)))]}
    df_item_list_exchange_same.append(df_item)
df_exchange_same=pd.DataFrame(df_item_list_exchange_same)
df_exchange_same

,id,total_xy,label,ch,new_total_xy,new_label,new_seq,error_type,error_tag
0,61c911a805bb4801b55d3692,"[[[23.503380988202366, 49.48923695937924], [23...","[竖, 横折, 横, 竖, 点, 卧钩, 点, 点]",忠,"[[[23.503380988202366, 49.48923695937924], [23...","[竖, 横折, 横, 竖, 点, 卧钩, 点, 点]","[0, 1, 2, 3, 6, 5, 4, 7]",[exchange],"[(exchange, same, (4, 6))]"
1,61c40c36c0766b5221b539cf,"[[[19.958181112802748, 55.52064526346544], [20...","[点, 撇, 撇, 点, 竖, 竖折/竖弯, 竖]",灿,"[[[29.401560153209658, 67.38995224774304], [29...","[点, 撇, 撇, 点, 竖, 竖折/竖弯, 竖]","[3, 1, 2, 0, 4, 5, 6]",[exchange],"[(exchange, same, (0, 3))]"
2,61c669dd05bb4801b55d3529,"[[[31.398977899130813, 49.86907140694566], [31...","[横, 撇, 横, 竖, 横, 点]",压,"[[[31.398977899130813, 49.86907140694566], [31...","[横, 撇, 横, 竖, 横, 点]","[0, 1, 4, 3, 2, 5]",[exchange],"[(exchange, same, (2, 4))]"
3,6188b1d587f3af50f2e803ff,"[[[38.601492272840176, 37.682988899751024], [3...","[撇, 横折钩, 撇, 点, 竖折/竖弯, 竖]",匈,"[[[40.895540685996565, 57.0051330075469], [41....","[撇, 横折钩, 撇, 点, 竖折/竖弯, 竖]","[2, 1, 0, 3, 4, 5]",[exchange],"[(exchange, same, (0, 2))]"
4,61c126e7c0766b5221b538f6,"[[[18.869901112164655, 46.12815801614298], [19...","[横, 竖, 竖, 横, 横, 提, 横撇/横钩, 点, 撇, 竖, 撇, 撇, 撇, 捺]",聚,"[[[18.869901112164655, 46.12815801614298], [19...","[横, 竖, 竖, 横, 横, 提, 横撇/横钩, 点, 撇, 竖, 撇, 撇, 撇, 捺]","[0, 1, 2, 3, 4, 5, 6, 7, 12, 9, 10, 11, 8, 13]",[exchange],"[(exchange, same, (8, 12))]"
...,...,...,...,...,...,...,...,...,...
195,6184a33f87f3af50f2e802cd,"[[[33.5285776727176, 52.362310325092395], [33....","[横, 横, 横]",三,"[[[33.850778710771095, 64.89173818748517], [33...","[横, 横, 横]","[1, 0, 2]",[exchange],"[(exchange, same, (0, 1))]"
196,606c0ed03717b67800102cf1,"[[[20.69377745317894, 53.10276235047668], [20....","[横折折折钩/横撇弯钩, 竖, 竖, 横折, 横, 横, 横]",阻,"[[[20.69377745317894, 53.10276235047668], [20....","[横折折折钩/横撇弯钩, 竖, 竖, 横折, 横, 横, 横]","[0, 1, 2, 3, 6, 5, 4]",[exchange],"[(exchange, same, (4, 6))]"
197,606c0ca3e00923b7467b5b07,"[[[23.999639841672888, 54.04104098182925], [23...","[横, 横, 撇折, 点, 横折钩, 撇]",动,"[[[14.092186155303295, 67.28851437969848], [14...","[横, 横, 撇折, 点, 横折钩, 撇]","[1, 0, 2, 3, 4, 5]",[exchange],"[(exchange, same, (0, 1))]"
198,6107dc34a11a58c6dbb96ba3,"[[[37.219309023717074, 39.18105245748012], [37...","[撇, 横, 横, 竖, 横, 竖]",年,"[[[37.219309023717074, 39.18105245748012], [37...","[撇, 横, 横, 竖, 横, 竖]","[0, 1, 4, 3, 2, 5]",[exchange],"[(exchange, same, (2, 4))]"


In [11]:
 error_df=pd.concat([df_more_split,df_more_nonsplit,df_less_merge,df_less_nonmerge,df_wrong_parallel,df_wrong_rotation,df_wrong_type,df_wrong_size,df_exchange_different,df_exchange_same]).reset_index(drop=True)

In [12]:
# error_df.to_csv("data/error_standard_squeezed.csv",index=False)

In [22]:
#随机森林需要的数据格式
trajectorys=[]
labels=[]
ids=[]
seqs=[]
chs=[]
error_ids=[]

for i in range(len(error_df)):
    item=error_df.iloc[i]
    mych=item.ch 
    myid=item.id
    for j in range(len(item.new_label)):
        myseq=j+1
        print(myid,myseq)
#         mytrajectory=eval(standard_df[(standard_df.id==myid)&(standard_df.seq==myseq)].trajectory.iloc[0])    
        mytrajectory=[ {'ab_x':k[0],'ab_y':k[1]}for k in item.new_total_xy[j]]
        
        mylabel=item.new_label[j]
        trajectorys.append(mytrajectory)
        labels.append(mylabel)
        ids.append(myid)
        seqs.append(myseq)
        chs.append(mych)
        error_ids.append(i)
error_df_format=pd.DataFrame([trajectorys,labels,ids,seqs,chs,error_ids]).T
error_df_format.columns=['trajectory','label','id','seq','ch','error_id']
error_df_format.to_csv("data/error_standard_format.csv",index=False)


61c00e25c0766b5221b538a8 1
61c00e25c0766b5221b538a8 2
61c00e25c0766b5221b538a8 3
61c00e25c0766b5221b538a8 4
61c00e25c0766b5221b538a8 5
61c00e25c0766b5221b538a8 6
61c00e25c0766b5221b538a8 7
61c00e25c0766b5221b538a8 8
61c00e25c0766b5221b538a8 9
61c00e25c0766b5221b538a8 10
61c00e25c0766b5221b538a8 11
61c00e25c0766b5221b538a8 12


IndexError: single positional indexer is out-of-bounds

In [23]:
standard_df[(standard_df.id==myid)&(standard_df.seq==myseq)]

,trajectory,label,id,seq,ch


In [20]:
eval(standard_df[(standard_df.id=='61c00e25c0766b5221b538a8')&(standard_df.seq==1)].trajectory.iloc[0]) 

[{'ab_x': 24.875433104029327,
  'ab_y': 46.55314158597832,
  'force': 92,
  'timeLong': 32106065000},
 {'ab_x': 25.113650317144106,
  'ab_y': 46.55314158597832,
  'force': 140,
  'timeLong': 32106065009},
 {'ab_x': 25.41142183353753,
  'ab_y': 46.55314158597832,
  'force': 180,
  'timeLong': 32106065018},
 {'ab_x': 25.530530440094907,
  'ab_y': 46.55314158597832,
  'force': 196,
  'timeLong': 32106065028},
 {'ab_x': 25.768747653209658,
  'ab_y': 46.976326328625376,
  'force': 256,
  'timeLong': 32106065037},
 {'ab_x': 26.126073472881785,
  'ab_y': 46.976326328625376,
  'force': 276,
  'timeLong': 32106065046},
 {'ab_x': 26.781170808947365,
  'ab_y': 47.45996603450773,
  'force': 316,
  'timeLong': 32106065056},
 {'ab_x': 26.959833718783443,
  'ab_y': 47.94360574039008,
  'force': 328,
  'timeLong': 32106065066},
 {'ab_x': 26.840725112226067,
  'ab_y': 48.30633551980185,
  'force': 332,
  'timeLong': 32106065076},
 {'ab_x': 27.138496628619492,
  'ab_y': 48.06451566686067,
  'force': 348

In [28]:
error_df_format=pd.read_csv("data/error_standard_format.csv")
error_df=pd.read_csv("data/error_standard_squeezed.csv")
error_df.error_type=error_df.error_type.apply(eval)
error_df.label=error_df.label.apply(eval)
error_df.error_tag=error_df.error_tag.apply(eval)
error_df['error_tag0']=error_df.error_tag.apply(lambda x: (x[0][0],x[0][1]))

In [5]:
#随机森林-抽取特征
#构造特征表
total_stroke=[]
total_simple_stroke=[]
width_list=[]
length_list=[]
ratio_list=[]
label_list=[]
dot_count_list=[]
dotdist_list=[]
angle_list=[]
data=error_df_format##########################################################
for k in range(len(data)):
    t=Stroke(data['trajectory'].iloc[k],data['label'].iloc[k])
    total_stroke.append(t)
    total_simple_stroke.append(t.simple_dot)
    dot_count_list.append(len(t.simple_dot))
    dotdist_list.append(t.dotdist_list)
#     if t.angle_list
    angle_list.append(t.angle_list)
    k=t.get_width_length_ratio()
    width_list.append(k[0])
    length_list.append(k[1])
    ratio_list.append(k[2])
    label_list.append(t.get_label())
dotdist_list_count=[len(i) for i in dotdist_list]
dotdist_list_max=[max(i) for i in dotdist_list]
dotdist_list_min=[min(i) for i in dotdist_list]
dotdist_list_median=[np.median(i) for i in dotdist_list] 
angle_list_max=[max(i) for i in angle_list]
angle_list_min=[min(i) for i in angle_list]
angle_list_median=[np.median(i) for i in angle_list] 
angle_list_90=[close90(i) for i in angle_list] 

E:\科研\规范汉字评价模型\code\Module\DotClass.py:143: RuntimeWarning: invalid value encountered in arccos
  arccos = np.arccos(vector_dot_product / (np.linalg.norm(v1) * np.linalg.norm(v2)))


In [6]:
error_df_rf=pd.DataFrame([label_list,data.id,width_list,length_list,ratio_list,dotdist_list_count,dotdist_list_max,dotdist_list_min,dotdist_list_median,angle_list_max,angle_list_min,angle_list_median,angle_list_90]).T
error_df_rf.columns=["label","id","width","height","ratio_w_h","dotdist_count","dotdist_max","dotdist_min","dotdist_median","angle_max","angle_min","angle_median","angle_90"]
error_df_rf=error_df_rf.fillna(0)
x_test_ml=error_df_rf[error_df_rf.columns[2:]]
y_test_ml=error_df_rf['label']
x_test_ml

,width,height,ratio_w_h,dotdist_count,dotdist_max,dotdist_min,dotdist_median,angle_max,angle_min,angle_median,angle_90
0,8.575820,21.038327,0.407628,9,15.845169,0.060455,1.245509,10.121267,0.030153,0.283114,79.878733
1,16.079662,6.891866,2.333136,12,13.755497,0.000000,0.203970,11.776613,0.000000,0.076606,78.223387
2,4.764344,26.116544,0.182426,7,21.332042,0.134781,1.654711,8.394532,0.013211,0.104011,81.605468
3,11.732198,14.871921,0.788882,6,6.880106,0.000000,3.741694,0.000000,0.000000,0.000000,86.172941
4,9.469134,4.594577,2.060937,5,6.716502,0.254585,1.211293,2.241514,0.014559,0.103485,87.758486
...,...,...,...,...,...,...,...,...,...,...,...
16048,1.786629,16.201930,0.110273,6,10.610213,0.120910,0.967234,4.895614,0.041252,0.330596,85.104386
16049,16.139216,5.138672,3.140737,11,14.506235,0.084862,0.238217,9.659187,0.001199,0.072445,80.340813
16050,13.042392,4.473667,2.915369,9,12.416084,0.000000,0.178663,7.862877,0.013973,0.000000,82.137123
16051,2.799052,16.201930,0.172760,7,12.482379,0.000000,1.023139,7.349703,0.000000,0.115461,82.650297


In [22]:
# 用随机森林模型预测结果
import pickle
#save model
f = open('model/rfc.pickle','rb')
rfc1 = pickle.load(f)
f.close()
rf_pred=rfc1.predict(x_test_ml)
test_ml_acc=accuracy_score(rf_pred,y_test_ml)
print('test acc:',test_ml_acc)


error_df_format_rf=copy.deepcopy(error_df_format)
error_df_format_rf['pred']=rf_pred
rf_res=error_df_format_rf.groupby(['error_id'])['pred'].apply(lambda x:list(x))
rf_res
#应用算法，查找错误
my_error_list=[]
includes=[]
equals=[]
mymap={"more":'多笔画',"less":'少笔画',"wrong":'笔画错误',"exchange":'笔顺错误'}

for i in range(len(rf_res)):
    item=error_df.iloc[i]
    string1=rf_res.iloc[i]
    string2=item['label']
    error=get_problems(string1,string2) 
    if mymap[item['error_type'][0]] in error:
        includes.append(1)
    else:
        includes.append(0)
    temp=[ mymap[item['error_type'][i]] for i in range(len(item['error_type']))]
    if temp == error:
        equals.append(1)
    else:
        equals.append(0)        
    my_error_list.append(error)
my_error_list




#dtw对于非原本字的类别搞不定



# 所以希望将两者结合去看。
#预测错误类别并查错


#单用dtw的模型结果

# 用dtw做分类错误校正，最后输出一个不错的结果

#改进的编辑距离算法去解决这个问题，为什么不能只用dtw去做这件事情#多类别错误可能来不及看qaq


test acc: 0.9283622998816421


[['多笔画', '笔画错误'],
 ['多笔画'],
 ['多笔画'],
 ['多笔画', '笔画错误'],
 ['多笔画'],
 ['多笔画'],
 ['多笔画', '笔画错误'],
 ['多笔画'],
 ['多笔画'],
 ['多笔画'],
 ['多笔画'],
 ['多笔画', '笔画错误'],
 ['多笔画', '笔画错误'],
 ['多笔画', '笔画错误'],
 ['多笔画', '笔画错误'],
 ['多笔画', '笔画错误'],
 ['多笔画'],
 ['多笔画', '笔画错误'],
 ['多笔画'],
 ['多笔画'],
 ['多笔画', '笔画错误'],
 ['多笔画'],
 ['多笔画'],
 ['多笔画', '笔画错误'],
 ['多笔画'],
 ['多笔画', '笔画错误'],
 ['多笔画', '笔画错误'],
 ['多笔画', '笔画错误'],
 ['多笔画', '笔画错误'],
 ['多笔画'],
 ['多笔画', '笔画错误'],
 ['多笔画'],
 ['多笔画'],
 ['多笔画', '笔画错误'],
 ['多笔画'],
 ['多笔画'],
 ['多笔画'],
 ['多笔画', '笔画错误'],
 ['多笔画'],
 ['多笔画', '笔画错误'],
 ['多笔画'],
 ['多笔画'],
 ['多笔画'],
 ['多笔画'],
 ['多笔画'],
 ['多笔画', '笔画错误'],
 ['多笔画', '笔画错误'],
 ['多笔画', '笔画错误'],
 ['多笔画'],
 ['多笔画', '笔画错误'],
 ['多笔画'],
 ['多笔画'],
 ['多笔画', '笔画错误'],
 ['多笔画', '笔画错误'],
 ['多笔画'],
 ['多笔画', '笔画错误'],
 ['多笔画'],
 ['多笔画', '笔画错误'],
 ['多笔画', '笔画错误'],
 ['多笔画'],
 ['多笔画'],
 ['多笔画', '笔画错误'],
 ['多笔画', '笔画错误'],
 ['多笔画'],
 ['多笔画'],
 ['多笔画', '笔画错误'],
 ['多笔画'],
 ['多笔画', '笔画错误'],
 ['多笔画', '笔画错误'],
 ['多笔画'],
 ['多笔画', '笔画错误'],
 ['多笔画', '笔画错误'],


In [32]:
e_res=pd.concat([error_df,pd.DataFrame(rf_res),pd.DataFrame(includes,columns=['includes']),pd.DataFrame(equals,columns=['equals'])],axis=1)
e_res.groupby('error_tag0').sum()/200
# e_res

,includes,equals
error_tag0,,
"(exchange, different)",0.275,0.275
"(exchange, same)",0.000,0.000
"(less, merge)",1.000,0.270
"(less, nonmerge)",1.000,0.900
"(more, nonsplit)",1.000,0.875
"(more, split)",1.000,0.565
"(wrong, parallel)",0.340,0.340
"(wrong, rotation)",0.710,0.710
"(wrong, size)",0.535,0.535


In [23]:
#只移动一笔，使其变得最像
sum(equals)

1091

In [ ]:
exit()

In [ ]:
#可视化其中一些笔画
plt.rcParams['axes.facecolor']='white'
plt.figure(1)     
plt.figure(figsize=(8,8))
plt.xlim(-10,100)
plt.ylim(10,120)
mycolor='black'
for i in range(len(t_total_xy[seq_idx])):
    plt.scatter(t_total_xy[seq_idx][i][0],t_total_xy[seq_idx][i][1],c='b')
for i in range(len(t_xy)):
    plt.scatter(t_xy[i][0],t_xy[i][1],c='r')
ax = plt.gca()                                 #获取到当前坐标轴信息
ax.xaxis.set_ticks_position('top')   #将X坐标轴移到上面
ax.invert_yaxis()                            #反转Y坐标轴
plt.axis('off') 
plt.show()
plt.close()

In [ ]:
一些SOTA分类，都不如规则在这个数据集上做得好。
fusion model。 

In [ ]:
#只是需要设计算法罢了


In [ ]:
# 在原始的分类基础上加上dtw校正
t_new_total_xy[0][0][0]=0 
t_new_total_xy
# df.iloc[idx_list_more_split[0]].total_xy[0][0][0]
# df_backup.iloc[idx_list_more_split[0]].total_xy[0][0][0]

In [ ]:
# 用传统模型，验证在单个数据集上的结果 



In [ ]:
import json 
with open('data/predict_topk_standard.json','r',encoding='utf8')as fp:
    json_data = json.load(fp)
    print('这是文件中的json数据：',json_data)
    print('这是读取到文件数据的数据类型：', type(json_data))
# json.load()

In [ ]:
model_res=pd.DataFrame(json_data)
# model_res.join(standard_df,on=['id','seq'],how='right')
# model_res

In [ ]:
#以整个汉字为单位组织
# 构造错误数据集 
standard_df1=standard_df.copy()
standard_df1

In [ ]:
standard_df1['key']=1 
standard_df1['num']=standard_df1.apply(lambda x:standard_df1[standard_df1.id==x.id].id.count(),axis=1)

In [ ]:
standard_df1['predict']=model_res.predict

In [ ]:
standard_df1['seq_real']=standard_df1.seq.copy()

In [ ]:
mymax=0
for j in range(len(standard_df1)):
    t=standard_df1.trajectory.iloc[j]
    xy=[]
    temp=len(eval(t))
    if temp>mymax:
        mymax=temp
print("单个笔画最多的采样点数：",mymax)

In [ ]:
#为了加入噪声，统计笔画前后采样点的差
total_xy=[]
for j in range(len(standard_df1)):
    t=standard_df1.trajectory.iloc[j]
    xy=[]
    for i in eval(t):
        xy.append([i['ab_x'],i['ab_y']])
    total_xy.append(xy)
standard_df1["total_xy"]=total_xy
standard_df1

In [ ]:
def get_mymap(k1=1 ,k2=2):
    mymap={}
    for i in range(25):
        if i==k1:
            mymap[i]=k2
        elif i==k2:
            mymap[i]=k1
        else:
            mymap[i]=i
    return mymap

In [ ]:
mystandard_df_list=[]
idlist=list(set(standard_df1.id))
# idlist=['6107cceea11a58c6dbb96b42']
# more，从所有笔画中随机，落笔开始的位置一致
for myid in idlist:
    standard_df2=standard_df1[standard_df1.id==myid].copy().reset_index(drop=True)
    standard_df2.seq=standard_df2.seq_real#换顺序
    myseq=random.randint(0,len(standard_df2)-1)
    myseq2=random.randint(0,len(standard_df2)-1)
    standard_df2['error_seq']=[myseq2+1 for i in range(len(standard_df2))]
    standard_df2['error_type']=['more' for i in range(len(standard_df2))]
    standard_df3=standard_df2.iloc[myseq]
    res=pd.concat([standard_df2.iloc[:myseq2],pd.DataFrame(standard_df3).T,standard_df2.iloc[myseq2:]],axis=0).reset_index(drop=True)
    res.seq=[i+1 for i in range(len(res))]
    mystandard_df_list.append(res)


    
#less，减少某一笔
for myid in idlist:
    standard_df2=standard_df1[standard_df1.id==myid].copy().reset_index(drop=True)
    standard_df2.seq=standard_df2.seq_real#换顺序
    myseq=random.randint(0,len(standard_df2)-1)
    standard_df2=standard_df2.drop(myseq)
    standard_df2['error_seq']=[myseq+1 for i in range(len(standard_df2))]
    standard_df2['error_type']=['less' for i in range(len(standard_df2))]
    standard_df2['key']=[2 for i in range(len(standard_df2))]
    mystandard_df_list.append(standard_df2)


#wrong,可以构造各类错误，平移。。。笔画错误。。。，换成非这个字这一笔的某个笔画
for myid in idlist:
    standard_df2=standard_df1[standard_df1.id==myid].copy().reset_index(drop=True)
    if len(standard_df2)<=1:
        continue
    standard_df2.seq=standard_df2.seq_real#换顺序
    myseq=random.randint(0,len(standard_df2)-1)
    myseq2=random.randint(0,len(standard_df2)-1)
    t=standard_df1[standard_df1.ch!=standard_df2.iloc[0].ch]
    myseq3=random.randint(0,len(t)-1)
    standard_df3=t.iloc[myseq3].copy()
    standard_df2=standard_df2.drop(myseq2)
    res=pd.concat([standard_df2.iloc[:myseq2],pd.DataFrame(standard_df3).T,standard_df2.iloc[myseq2:]],axis=0).reset_index(drop=True)
    res.seq=[i+1 for i in range(len(res))]
    res['error_seq']=[myseq2+1 for i in range(len(res))]
    res['error_type']=['wrong' for i in range(len(res))]
    res['key']=[3 for i in range(len(res))]
#     res['num']=[len(res) for i in range(len(res))]
    mystandard_df_list.append(res)


# exchange
flag=4
dataframe2=standard_df1.copy().reset_index(drop=True)
for k1 in range(1,23):
    for k2 in range(k1+1,24):
        dataframe2.seq=dataframe2.seq_real.map(get_mymap(k1 ,k2))#换顺序
        dataframe2['error_seq']=[[k1,k2] for i in range(len(dataframe2))]
        dataframe2['error_type']=['exchange' for i in range(len(dataframe2))]
        standard_df2=dataframe2[dataframe2.num>=max(k1,k2)].copy()#只有有这么多的笔画才能实现交换
        standard_df2['key']=[flag for i in range(len(standard_df2))]
        flag+=1
        mystandard_df_list.append(standard_df2)


In [ ]:
res_df=pd.concat(mystandard_df_list).reset_index(drop=True)
res_df

In [ ]:
#gooddf,1674个汉字
chlist=list(set(standard_df1['ch']))
temp_df_list=[]
for ch in chlist:
    myid=standard_df1[standard_df1.ch==ch].iloc[0].id
    temp_df=standard_df1[standard_df1.id==myid]
    temp_df_list.append(temp_df)
gooddf_all=pd.concat(temp_df_list).reset_index(drop=True)
gooddf_all=gooddf_all[['id','key','ch','num','seq','seq_real','total_xy']].reset_index(drop=True)
gooddf_all

In [ ]:
gooddf_all_2=gooddf_all[['ch','seq','total_xy']].copy()
gooddf_all_2['standard_xy']=gooddf_all_2.total_xy
gooddf_all_2=gooddf_all_2.drop('total_xy',axis=1)
gooddf_all_2

In [ ]:
exchange_df=res_df[res_df.error_type=='exchange'].copy()
exchange_df=exchange_df.sort_values(['id','key','seq'])
exchange_df=exchange_df[['id','key','ch','num','seq','seq_real','total_xy']].reset_index(drop=True)
#做一个连接
exchange_df=exchange_df.set_index(['ch','seq']).join(gooddf_all_2.set_index(['ch','seq'])).reset_index().sort_values(['id','key','seq'])
exchange_df

In [ ]:
# #先把首尾的符号加入【0，0】【1，1】,新版代码暂时不用添加
# #res_df,id+key唯一标识
# id_1=[]
# key_1=[]
# ch_1=[]
# num_1=[]
# seq_1=[]
# seq_real_1=[]
# total_xy_1=[]
# standard_xy_1=[]


# temp=exchange_df[['id','key','ch','num']].drop_duplicates().reset_index(drop=True).copy()
# for i in range(len(temp)):
#     t=temp.iloc[i]
#     for j in range(t.num+1,30+1):
#         id_1.append(t.id)
#         key_1.append(t.key)
#         ch_1.append(t.ch)
#         num_1.append(t.num)
#         seq_1.append(j)
#         seq_real_1.append(0)
#         total_xy_1.append([[0,0]])
#         standard_xy_1.append([[0,0]])
    
# temp1=pd.DataFrame([id_1,key_1,seq_1,seq_real_1,total_xy_1,standard_xy_1]).T
# cols=['id','key','seq','seq_real','total_xy','standard_xy']
# temp1.columns=cols
# exchange_df_new=pd.concat([exchange_df[cols],temp1]).sort_values(['id','key','seq']).reset_index(drop=True).copy()
# # exchange_df_new=pd.concat([exchange_df,temp1]).sort_values(['id','key','seq']).reset_index(drop=True).copy()
# import random
# random.seed(1)
# N=12000
# t=exchange_df_new[['id','key']].drop_duplicates().reset_index(drop=True)
# idxlist=random.sample(range(0,  len(t)), N)
# t1=t.iloc[idxlist[:10000]]
# t2=t.iloc[idxlist[10000:]]
# exchange_df_new1=t1.set_index(['id','key']).join(exchange_df_new.set_index(['id','key'])).reset_index().sort_values(['id','key','seq']).copy()
# exchange_df_new2=t2.set_index(['id','key']).join(exchange_df_new.set_index(['id','key'])).reset_index().sort_values(['id','key','seq']).copy()
# exchange_df_new2

In [ ]:
#根据id和key合并,此处弃用
#随机划分训练集和测试集
total_xy=exchange_df.groupby(['id','key'])['total_xy'].apply(lambda x:list(x))
standard_xy=exchange_df.groupby(['id','key'])['standard_xy'].apply(lambda x:list(x))
seq_real=exchange_df.groupby(['id','key'])['seq_real'].apply(lambda x:list(x))
random.seed(1)
N=12000
t=total_xy.to_frame().join(standard_xy.to_frame()).join(seq_real.to_frame())
# idxlist=random.sample(range(0,  len(t)), N)
# t1=t.iloc[idxlist[:10000]]
# t2=t.iloc[idxlist[10000:]]
# exchange_df_new1=t1.copy()
# exchange_df_new2=t2.copy()
# exchange_df_new2
# exchange_df_new1.to_csv("data/model_data_train.csv",index=False) 
# exchange_df_new2.to_csv("data/model_data_test.csv",index=False) 

In [ ]:
t=t.reset_index()
idlist_total=list(set(standard_df.id))
idlist_test=random.sample(idlist_total, len(idlist_total)//5)
idlist_train=[i for i in idlist_total if i not in idlist_test]
t1=t[t.id.isin(idlist_train)]
t2=t[t.id.isin(idlist_test)]
exchange_df_new1=t1.copy()
exchange_df_new2=t2.copy()
exchange_df_new1.to_csv("data/model_data_train_1.csv",index=False) 
exchange_df_new2.to_csv("data/model_data_test_1.csv",index=False) 

In [ ]:
print('train dataset:',len(exchange_df_new1),'est dataset:',len(exchange_df_new2))

In [ ]:
exchange_df_new1


In [ ]:
#先把首尾的符号加入【0，0】【1，1】,新版代码暂时不用添加
#res_df,id+key唯一标识
seq_1=[]
seq_real_1=[]
total_xy_1=[]
standard_xy_1=[]
seq_2=[]
seq_real_2=[]
total_xy_2=[]
standard_xy_2=[]

temp=exchange_df[['id','key','ch','num']].drop_duplicates().reset_index(drop=True).copy()
for i in range(len(temp)):
    t=temp.iloc[i]
    seq_1.append(0)
    seq_real_1.append(0)
    total_xy_1.append([[0,0]])
    standard_xy_1.append([[0,0]])
    seq_2.append(t.num+1)
    seq_real_2.append(0)
    total_xy_2.append([[1,1]])
    standard_xy_2.append([[1,1]])
    
temp1=temp
temp2=temp.copy()
temp1['seq']=seq_1
temp1['seq_real']=seq_real_1
temp1['total_xy']=total_xy_1
temp1['standard_xy']=standard_xy_1
temp2['seq']=seq_2
temp2['seq_real']=seq_real_2
temp2['total_xy']=total_xy_2
temp2['standard_xy']=standard_xy_2
exchange_df_new=pd.concat([exchange_df,temp1,temp2]).sort_values(['id','key','seq']).reset_index(drop=True).copy()

In [ ]:
exchange_df_new

In [ ]:
#选取其中部分存储,82068个字，随机抽取1000个字
import random
random.seed(1)
N=12000
t=exchange_df_new[['id','key']].drop_duplicates().reset_index(drop=True)
idxlist=random.sample(range(0,  len(t)), N)
t1=t.iloc[idxlist[:10000]]
t2=t.iloc[idxlist[10000:]]
exchange_df_new1=t1.set_index(['id','key']).join(exchange_df_new.set_index(['id','key'])).reset_index().sort_values(['id','key','seq']).copy()
exchange_df_new2=t2.set_index(['id','key']).join(exchange_df_new.set_index(['id','key'])).reset_index().sort_values(['id','key','seq']).copy()
# exchange_df_new.iloc[idxlist][]  
# t.set_index(['ch','seq'])
exchange_df_new2

In [ ]:
# exchange_df_new 
# exchange_df_new1[['total_xy','standard_xy','seq_real']].to_csv("data/model_data_train.csv",index=False) 
# exchange_df_new2[['total_xy','standard_xy','seq_real']].to_csv("data/model_data_test.csv",index=False) 

In [ ]:
def add_key(datawork):
    keylist=[]
    m={}
    for i in range(len(datawork)):
        t=datawork.iloc[i]
        if m.get(t.id) ==None:
            m[t.id]=0
        if i==0 or (i>0 and  datawork.iloc[i-1].seq>=t.seq):
            m[t.id]+=1
        keylist.append(m[t.id])
    datawork["key"]=keylist
    return datawork
# t=add_key(res_df)
# t 

In [ ]:
import numpy as np

def levenshtein_distance(string1, string2):
    n1 = len(string1)
    n2 = len(string2)
    return _levenshtein_distance_matrix(string1, string2)[n1, n2]

def damerau_levenshtein_distance(string1, string2):
    n1 = len(string1)
    n2 = len(string2)
    return _levenshtein_distance_matrix(string1, string2, True)[n1, n2]

def get_ops(string1, string2, is_damerau=False):
    dist_matrix = _levenshtein_distance_matrix(string1, string2, is_damerau=is_damerau)
    i, j = dist_matrix.shape
    i -= 1
    j -= 1
    ops = list()
    while i != -1 and j != -1:
        if is_damerau:
            if i > 1 and j > 1 and string1[i-1] == string2[j-2] and string1[i-2] == string2[j-1]:
                if dist_matrix[i-2, j-2] < dist_matrix[i, j]:
#                     ops.insert(0, ('transpose', i - 1, i - 2))
                    ops.insert(0, ( i - 1, i - 2,string1[i-1],string1[i-2],'transpose'))
                    i -= 2
                    j -= 2
                    continue
        index = np.argmin([dist_matrix[i-1, j-1], dist_matrix[i, j-1], dist_matrix[i-1, j]])
        
        if index == 0:
            if dist_matrix[i, j] > dist_matrix[i-1, j-1]:
#                 ops.insert(0, ('replace', i - 1, j - 1))
                ops.insert(0, ( i - 1, j - 1,string1[i-1],string2[j-1],'replace'))
            i -= 1
            j -= 1
        elif index == 1:
#             ops.insert(0, ('insert', i - 1, j - 1))
            ops.insert(0, ( i - 1, j - 1,string1[i-1],string2[j-1],'insert'))
            j -= 1
        elif index == 2:
#             ops.insert(0, ('delete', i - 1, i - 1))
            ops.insert(0, ( i - 1, i - 1,string1[i-1],string1[i-1],'delete'))
            i -= 1
    return ops

def execute_ops(ops, string1, string2):
    strings = [string1]
    string = list(string1)
    shift = 0
    for op in ops:
        i, j = op[1], op[2]
        if op[0] == 'delete':
            del string[i + shift]
            shift -= 1
        elif op[0] == 'insert':
            string.insert(i + shift + 1, string2[j])
            shift += 1
        elif op[0] == 'replace':
            string[i + shift] = string2[j]
        elif op[0] == 'transpose':
            string[i + shift], string[j + shift] = string[j + shift], string[i + shift]
        strings.append(''.join(string))
    return strings

def _levenshtein_distance_matrix(string1, string2, is_damerau=False):
    n1 = len(string1)
    n2 = len(string2)
    d = np.zeros((n1 + 1, n2 + 1), dtype=int)
    for i in range(n1 + 1):
        d[i, 0] = i
    for j in range(n2 + 1):
        d[0, j] = j
    for i in range(n1):
        for j in range(n2):
            if string1[i] == string2[j]:
                cost = 0
            else:
                cost = 1
            d[i+1, j+1] = min(d[i, j+1] + 1, # insert
                              d[i+1, j] + 1, # delete
                              d[i, j] + cost) # replace
            if is_damerau:
                if i > 0 and j > 0 and string1[i] == string2[j-1] and string1[i-1] == string2[j]:
                    d[i+1, j+1] = min(d[i+1, j+1], d[i-1, j-1] + cost) # transpose
    return d


def get_score(resultdf,idx,chdf,is_damerau=True):
    t=resultdf.iloc[idx]
    string1=t['label']
    string2=chdf[chdf.ch==t['ch']].iloc[0].label
    word1=string1
    word=string2
    
#     print(resultdf.iloc[idx]['label'])
#     print(chdf[chdf.ch==t['ch']].iloc[0].label)
#     print(string1,string2)
    dist_matrix = _levenshtein_distance_matrix(string1, string2, is_damerau=is_damerau)
    ops = get_ops(string1, string2, is_damerau=is_damerau)
    res= len(ops)
    basescore=(len(word)+len(word1)-res)/(len(word)+len(word1))#莱温斯坦比
    predict=t['predict']
    scorelist=[i[0]['logit'] for i in predict]
    adjscore=np.mean(scorelist)
#     print(0.9*basescore+0.1*adjscore)
    return 0.9*basescore+0.1*adjscore
    
    
#     print("你的分数是：",round(score*100,1))
    
# get_score(resultdf,0,chdf)
def get_problems(string1,string2,is_damerau=True):
    problems=[]
    dist_matrix = _levenshtein_distance_matrix(string1, string2, is_damerau=is_damerau)
#         print(dist_matrix)
    ops = get_ops(string1, string2, is_damerau=is_damerau)
    if len(string1)-len(string2)>0:
        problems.append("多笔画")
    elif len(string1)-len(string2)<0:
        problems.append("少笔画")
    result=ops
    comments=[]
    for i in result:
        if i[4]=="insert":
            t="增加"
            comment="你在第"+str(i[0]+1)+"笔后要"+t+i[3]
        elif i[4]=="delete":
            t="删除"
            comment="你的第"+str(i[0]+1)+"笔（"+i[2]+"）要"+t

        elif i[4]=="replace":
            t="替换"
            comment="【笔画错误】你的第"+str(i[0]+1)+"笔（"+i[2]+"）要"+t+"成"+i[3]
            if "笔画错误" not in problems:
                problems.append("笔画错误")
        else:
            t="交换"
            comment="【笔顺错误】你的第"+str(i[0]+1)+"笔（"+i[2]+"）要和"+"第"+str(i[1]+1)+"笔（"+i[3]+"）"+t
            if "笔顺错误" not in problems:
                problems.append("笔顺错误")
        comments.append(comment)
    return problems
def get_comments(string1,string2,is_damerau=True):
    problems=[]
    dist_matrix = _levenshtein_distance_matrix(string1, string2, is_damerau=is_damerau)
#         print(dist_matrix)
    ops = get_ops(string1, string2, is_damerau=is_damerau)
    if len(string1)-len(string2)>0:
        problems.append("多笔画")
    elif len(string1)-len(string2)<0:
        problems.append("少笔画")
    result=ops
    comments=[]
    for i in result:
        if i[4]=="insert":
            t="增加"
            comment="你在第"+str(i[0]+1)+"笔后要"+t+i[3]
        elif i[4]=="delete":
            t="删除"
            comment="你的第"+str(i[0]+1)+"笔（"+i[2]+"）要"+t

        elif i[4]=="replace":
            t="替换"
            comment="【笔画错误】你的第"+str(i[0]+1)+"笔（"+i[2]+"）要"+t+"成"+i[3]
            if "笔画错误" not in problems:
                problems.append("笔画错误")
        else:
            t="交换"
            comment="【笔顺错误】你的第"+str(i[0]+1)+"笔（"+i[2]+"）要和"+"第"+str(i[1]+1)+"笔（"+i[3]+"）"+t
            if "笔顺错误" not in problems:
                problems.append("笔顺错误")
        comments.append(comment)
    return comments
def get_problems_and_comments(string1,string2,is_damerau=True):
    problems=[]
    dist_matrix = _levenshtein_distance_matrix(string1, string2, is_damerau=is_damerau)
#         print(dist_matrix)
    ops = get_ops(string1, string2, is_damerau=is_damerau)
    if len(string1)-len(string2)>0:
        problems.append("多笔画")
    elif len(string1)-len(string2)<0:
        problems.append("少笔画")
    result=ops
    comments=[]
    cnn_res=[]
    for i in result:
        if i[4]=="insert":
            t="增加"
            comment="你在第"+str(i[0]+1)+"笔后要"+t+i[3]
            cnn_res.append([i[0]+1,'less'])
        elif i[4]=="delete":
            t="删除"
            if i[0]==-1:
                continue
            comment="你的第"+str(i[0]+1)+"笔（"+i[2]+"）要"+t
            cnn_res.append([i[0]+1,'more'])
        elif i[4]=="replace":
            t="替换"
            comment="【笔画错误】你的第"+str(i[0]+1)+"笔（"+i[2]+"）要"+t+"成"+i[3]
            if "笔画错误" not in problems:
                problems.append("笔画错误")
            cnn_res.append([i[0]+1,'wrong'])            
        else:
            t="交换"
            comment="【笔顺错误】你的第"+str(i[0]+1)+"笔（"+i[2]+"）要和"+"第"+str(i[1]+1)+"笔（"+i[3]+"）"+t
            if "笔顺错误" not in problems:
                problems.append("笔顺错误")
            cnn_res.append([[i[0]+1,i[1]+1],'exchange'])
        comments.append(comment)
    return problems,comments,cnn_res
    


In [ ]:
def get_resultdf(res_df,myid=None):
    if myid!=None:
        df=res_df[res_df.id==myid].copy().reset_index(drop=True)
#         df=add_key(df)
    else:
        df=res_df.copy()
#         df=add_key(df)
    labellist=[]
    idlist=[]
    keylist=[]
    chlist=[]
    predictlist=[]
    cnt=[]
    cntpred=[]
    error_seq=[]
    error_type=[]
#     last_id="-1"
#     last_key=-1
    for i in range(len(df)):
        if i==0:
            t=df.iloc[i].predict[0]['label']
            cnt=[t]
            cntpred=[df.iloc[i].predict]
            continue
#         elif(df.iloc[i].id!=df.iloc[i-1].id or df.iloc[i].seq_real<=df.iloc[i-1].seq_real) :
        elif(df.iloc[i].id!=df.iloc[i-1].id or df.iloc[i].key!=df.iloc[i-1].key) :
            idlist.append(df.iloc[i-1].id)
            keylist.append(df.iloc[i-1].key)
            chlist.append(df.iloc[i-1].ch)
            error_seq.append(df.iloc[i-1].error_seq)
            error_type.append(df.iloc[i-1].error_type)
    #         predictlist.append(df.iloc[i].predict)
            labellist.append(cnt)
            predictlist.append(cntpred)
            t=df.iloc[i].predict[0]['label']
            cnt=[t]
            cntpred=[df.iloc[i].predict]
        elif(i==len(df)-1):
            idlist.append(df.iloc[i].id)
            keylist.append(df.iloc[i].key)
            chlist.append(df.iloc[i-1].ch)
            error_seq.append(df.iloc[i-1].error_seq)
            error_type.append(df.iloc[i-1].error_type)
            cnt.append(df.iloc[i].predict[0]['label'])
            cntpred.append(df.iloc[i].predict)
            labellist.append(cnt)
            predictlist.append(cntpred)
        else:
            cnt.append(df.iloc[i].predict[0]['label'])
            cntpred.append(df.iloc[i].predict)

    #整理成合适的格式
    resultdf=pd.DataFrame({'ch':chlist,'label':labellist,'id':idlist,'key':keylist,'predict':predictlist,'error_seq':error_seq,'error_type':error_type})
    return resultdf

def get_remarkdf(res_df,myid=None):
    chdf=pd.read_csv("data/words-new-2.csv")
    chdf['ch']=chdf['汉字']
    chdf['label']=chdf['笔画'].apply(lambda x:x.split("、"))
    resultdf=get_resultdf(res_df,myid)
    scorelist=[]
    problemslist=[]
    commentslist=[]
    cnn_res_list=[]
    is_damerau=True
    for idx in range(len(resultdf)):
        try:
            t=resultdf.iloc[idx]
            string1=t['label']
            string2=chdf[chdf.ch==t['ch']].iloc[0].label
            dist_matrix = _levenshtein_distance_matrix(string1, string2, is_damerau=is_damerau) #后面函数中要用到
#             print(string1,string2)
            problems,comments,cnn_res=get_problems_and_comments(string1,string2,is_damerau=True)
#             print(1)
            score=get_score(resultdf,idx,chdf,is_damerau=True)
            scorelist.append(score)
            problemslist.append(problems) 
            commentslist.append(comments)
            cnn_res_list.append(cnn_res)
        except:
            scorelist.append(1)
            problemslist.append([]) 
            commentslist.append([])
            cnn_res_list.append([])
            
    resultdf['score']=scorelist
    resultdf['problems']=problemslist
    resultdf['comments']=commentslist
    resultdf['cnn_res']=cnn_res_list
    
    
    return resultdf

In [ ]:
idlist

In [ ]:
# res_df0=res_df.sort_values(['id','key','seq']).copy().reset_index(drop=True)
# t_df_list=[]
# for i in range(100):
#     myid=idlist[i]
#     t_df=get_remarkdf(res_df0,myid)
#     t_df_list.append(t_df)
t_df=pd.concat(t_df_list).reset_index(drop=True)
cnn_res_list=list(t_df.cnn_res)
error_seq_list=list(t_df.error_seq)
error_type_list=list(t_df.error_type)
evaluation=[]
for i in range(len(t_df)):
    flag=0
    if len(cnn_res_list[i])!=1:
#         print(1)
        evaluation.append(0)
        continue
    t=cnn_res_list[i][0]
    if t[0]==error_seq_list[i] and t[1]==error_type_list[i]:
        evaluation.append(1) 
#         continue
    elif type(t[0])==type([]) and type(error_seq_list[i])==type([]):
        if sorted(t[0])==sorted(error_seq_list[i]) and  t[1]==error_type_list[i]  and flag!=1:
            evaluation.append(1) 
    else:
        evaluation.append(0) 
        
        
t_df['evaluation']=evaluation  
t_df
# resultdf=get_resultdf(res_df,myid)
# resultdf
# res_df
# res_df[res_df.id=='6107cceea11a58c6dbb96b42']
# myid='6107cceea11a58c6dbb96b42'
# df=res_df[res_df.id==myid].copy().reset_index(drop=True)
# df=add_key(df)

In [ ]:
# t_df=pd.concat(t_df_list).reset_index(drop=True)
cnn_res_list=list(t_df.cnn_res)
error_seq_list=list(t_df.error_seq)
error_type_list=list(t_df.error_type)
evaluation=[]
for i in range(len(t_df)):
    flag=0
    for j in range(len(cnn_res_list[i])):
        t=cnn_res_list[i][j]
        if type(t[0])==type([]) and type(error_seq_list[i])==type([]) :
            if sorted(t[0])==sorted(error_seq_list[i]) and  t[1]==error_type_list[i]  and flag!=1:
#             evaluation.append(1) 
                flag=1
        else:
            if (t[0]==error_seq_list[i] and t[1]==error_type_list[i] ) and flag!=1:
    #             evaluation.append(1) 
                flag=1
    
    evaluation.append(flag)
        
        
t_df['evaluation2']=evaluation  
t_df

In [ ]:
# t[0]
# sorted(t[0])
# type(t[0])==type([]) 
# sorted(error_seq_list[i])
# sorted(t[0])
# sorted(error_seq_list[i])
error_seq_list[i]

In [ ]:
t_df.groupby(['error_type','evaluation']).describe()

In [ ]:
t_df.groupby(['error_type','evaluation2']).describe()